In [ ]:
import pandas as pd

# Load the data from Excel file
data = pd.read_csv('_annotations.csv')

# Display the first few rows of the dataframe
print(data.head())

                                            filename  width  height   class  \
0  traffic_img1137_jpg.rf.3c89100f1b848e752675c2e...    640     640  severe   
1  images_jpg.rf.3c44f7990f01238a95582330f4c6d4e0...    640     640  severe   
2  Semi-RealisticVD-20_jpg.rf.3cbb4c9a36a57467c8f...    640     640  severe   
3  Semi-RealisticVD-20_jpg.rf.3cbb4c9a36a57467c8f...    640     640  severe   
4  sev11_jpg.rf.3c6908705e3752dc9cbdea964b3e1cfb.jpg    640     640  severe   

   xmin  ymin  xmax  ymax  
0   383     0   640   574  
1   211   147   623   560  
2   201   175   313   284  
3   283   168   380   265  
4   103    47   627   344  


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def preprocess_image(file_name, target_size=(640, 640)):
    image = cv2.imread(file_name)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

# Apply preprocessing to each image
images = [preprocess_image(f'path_to_images/{fname}') for fname in data['Name of the file']]
images = np.array(images)

# Prepare labels
labels = data['severity of the accident'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(640, 640, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification for severity
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
def process_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    predictions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        preprocessed_frame = preprocess_image(frame)
        prediction = model.predict(np.expand_dims(preprocessed_frame, axis=0))
        predictions.append(prediction)

    cap.release()
    return predictions

# Example usage
video_predictions = process_video('path_to_video.mp4', model)


In [ ]:
# Assuming the Excel sheet has columns 'max_x', 'min_x', 'max_y', 'min_y' for bounding boxes
bbox_data = data[['max_x', 'min_x', 'max_y', 'min_y']].values


In [ ]:
def process_video_with_bounding_boxes(video_path, model, bbox_data):
    cap = cv2.VideoCapture(video_path)
    out = None
    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        preprocessed_frame = preprocess_image(frame)
        prediction = model.predict(np.expand_dims(preprocessed_frame, axis=0))

        if prediction[0][0] >= 0.5:  # Assuming a threshold of 0.5 for accident detection
            bbox = bbox_data[frame_index]
            cv2.rectangle(frame, (int(bbox[1]), int(bbox[3])), (int(bbox[0]), int(bbox[2])), (0, 255, 0), 2)

        if out is None:
            # Initialize video writer
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()

# Example usage
process_video_with_bounding_boxes('path_to_video.mp4', model, bbox_data)
